# Insights

In [1]:
import os
import core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### **Analysis Setup**

To explore the data, we must first pull all of the scraped data into "nice" tabular forms

In [11]:
# specify the paths to data files
problems_path = os.path.join("data","problems.json") 
hold_setup_base = "-hold-setup.json"
holdsetup_path_2020_mini = os.path.join("data","mini-moonboard-2020"+hold_setup_base)
holdsetup_path_2020 = os.path.join("data","moonboard-2020"+hold_setup_base)
holdsetup_path_2019 = os.path.join("data","moonboard-masters-2019"+hold_setup_base)
holdsetup_path_2017 = os.path.join("data","moonboard-masters-2017"+hold_setup_base)

# extract data from file
problems = core.import_json(problems_path)
holds_mini = core.import_json(holdsetup_path_2020_mini)
holds_2020 = core.import_json(holdsetup_path_2020)
holds_2019 = core.import_json(holdsetup_path_2019)
holds_2017 = core.import_json(holdsetup_path_2017)

In [130]:
# convert holdset json objects to tabular dataframe
setups = {"Mini-Moonboard-2020":holds_mini,"Moonboard-2020":holds_2020,"2019-Masters":holds_2019,"2017-Masters":holds_2017}
hold_df = pd.DataFrame()
for name,setup in setups.items():
    df_set = pd.DataFrame()
    for holdset in setup:
        # unroll nested dictionaries per holdset in setup in setups
        df1 = pd.DataFrame.from_dict(holdset).rename(columns={"Id":"HoldsetId","Description":"HoldsetName","Color":"HoldsetColor"}).drop(columns=["Holds"])
        df2 = pd.DataFrame.from_dict(holdset["Holds"]).rename(columns={"Id":"HoldId","Number":"HoldNumber"}).drop(columns=["Location","HoldsetDescription"])
        df3 = pd.DataFrame.from_records(pd.DataFrame.from_dict(holdset["Holds"])["Location"]).rename(columns={
            "Description":"LocationCode","DirectionString":"DirectionCode"}).drop(columns=[
                "Id","Holdset","Color","Type","HoldNumber"])
        # concatenate the unrolled data together into one dataframe, add shared feature, and append to master dataframe
        con = pd.concat([df1,df2,df3],axis=1)
        con["BoardSetup"] = name
        hold_df = pd.concat([hold_df,con])

In [131]:
print(hold_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656 entries, 0 to 31
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   HoldsetId      656 non-null    int64 
 1   HoldsetName    656 non-null    object
 2   HoldsetColor   656 non-null    object
 3   HoldId         656 non-null    int64 
 4   HoldNumber     656 non-null    object
 5   HoldType       656 non-null    int64 
 6   LocationCode   656 non-null    object
 7   X              656 non-null    int64 
 8   Y              656 non-null    int64 
 9   Rotation       656 non-null    int64 
 10  Direction      656 non-null    int64 
 11  DirectionCode  656 non-null    object
 12  BoardSetup     656 non-null    object
dtypes: int64(7), object(6)
memory usage: 71.8+ KB
None


In [128]:
# convert problems json object to tabular dataframe
df1 = pd.DataFrame.from_dict(problems,orient="index").drop(columns=["UserGrade","MoonBoardConfiguration","Setter",
    "MoonBoardConfigurationId","FirstAscender","Rating","NumberOfTries","NameForUrl","ApiId","DateInserted","DateUpdated",
    "DateDeleted","Holdsetup","Moves","Locations","Attempts","ProblemType","Holdsets","RepeatText","IsMaster",
    "IsAssessmentProblem"]).rename(columns={"DateTimeString":"RouteRecordDate","Id":"RouteId","UserRating":"RouteRating",
    "Grade":"RouteGrade","Name":"RouteName","Method":"RouteMethod","Repeats":"RouteRepeats"}).reset_index(drop=True)
df2 = pd.DataFrame.from_records(pd.DataFrame.from_dict(problems,orient="index")["MoonBoardConfiguration"]).drop(columns=[
    "LowGrade","HighGrade"]).rename(columns={"Id":"ConfigurationId","Description":"ConfigurationName"}).reset_index(drop=True)
df3 = pd.DataFrame.from_records(pd.DataFrame.from_dict(problems,orient="index")["Setter"]).drop(columns=["ProfileImageUrl",
    "Firstname","Lastname","CanShareData"]).rename(columns={"Id":"SetterId","Nickname":"SetterName","City":"SetterCity",
    "Country":"SetterCountry"}).reset_index(drop=True)
problem_df = pd.concat([df1,df2,df3],axis=1)

In [132]:
print(problem_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390 entries, 0 to 1389
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   RouteMethod        1390 non-null   object
 1   RouteName          1390 non-null   object
 2   RouteGrade         1390 non-null   object
 3   RouteRating        1390 non-null   int64 
 4   RouteRepeats       1390 non-null   int64 
 5   IsBenchmark        1390 non-null   bool  
 6   Upgraded           1390 non-null   bool  
 7   Downgraded         1390 non-null   bool  
 8   RouteId            1390 non-null   int64 
 9   RouteRecordDate    1390 non-null   object
 10  ConfigurationId    1390 non-null   int64 
 11  ConfigurationName  1390 non-null   object
 12  SetterId           1390 non-null   object
 13  SetterName         1390 non-null   object
 14  SetterCity         1390 non-null   object
 15  SetterCountry      1390 non-null   object
dtypes: bool(3), int64(4), object(9)
memory usa

In [129]:
# convert moves embedded in problems json object to tabular dataframe
df4 = pd.DataFrame.from_dict(pd.DataFrame.from_dict(problems,orient="index")["Moves"]).reset_index(drop=True)
move_df = pd.DataFrame()
i = 0
for moves in df4["Moves"]:
    df5 = pd.DataFrame.from_records(moves).rename(columns={"Id":"MoveId"})
    df5["RouteId"] = df1.at[i,"RouteId"]
    move_df = pd.concat([move_df,df5])
    i += 1

In [133]:
print(move_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8876 entries, 0 to 5
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MoveId       8876 non-null   int64 
 1   Description  8876 non-null   object
 2   IsStart      8876 non-null   bool  
 3   IsEnd        8876 non-null   bool  
 4   RouteId      8876 non-null   int64 
dtypes: bool(2), int64(2), object(1)
memory usage: 294.7+ KB
None


### **Exploratory Data Analysis**
Now that we have formatted the data "nicely", we can begin the process of identifying patterns which might be indicative of collinear relationships